In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv('data.csv')

In [4]:
raw_data.drop(raw_data[raw_data['Tp_est'] == 0.0].index, inplace=True);

Separado o dataset em treinamento e validação, além de separar o dataset de treinamento por estação.

In [19]:
# train_test_group = all_data.groupby(all_data['time'].str.contains('2022'))

# train_data = train_test_group.get_group(False)
# test_data = train_test_group.get_group(True)

# train_data_by_est = train_data.groupby(['Est'])

In [5]:
# print(train_data_by_est.groups.keys())
print(len(raw_data))
print(len(data))

115172


TypeError: object of type 'NoneType' has no len()